In [2]:
! pip install langchain tiktoken openai pypdf faiss-cpu huggingface_hub python-docx selenium unstructured -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 75.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 24.9 MB/s eta 0:00:0

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from docx import Document
from langchain.document_loaders import SeleniumURLLoader

In [47]:
# Load the PDF file. English or French only

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=64,
    separators=['\n\n', '\n', '(?=>\. )', ' ', ''])

pdf_doc_path='/content/drive/MyDrive/Data S/data/1594383154-world-war-ii.pdf'
if pdf_doc_path:
    loader = PyPDFLoader(pdf_doc_path)
    pages = loader.load_and_split()

txt_path=''
if txt_path:
    loader=TextLoader(txt_path)
    pages=loader.load()

docx_path=''
if docx_path:
  word_doc=Document(docx_path)
  passages=[p.text for p in word_doc.paragraphs]
  content = "\n".join(passages)
  file_txt_path="./file.txt"
  with open(file_txt_path, "w") as f:
    f.write(content)
  loader=TextLoader(file_txt_path)
  pages=loader.load()

url=['']
if url[0]:
  loader = SeleniumURLLoader(url)
  pages=loader.load()

texts = text_splitter.split_documents(pages)
embeddings = OpenAIEmbeddings(openai_api_key="")

In [48]:
vector_db = FAISS.from_documents(texts, embeddings)
vector_db.save_local("./FAISS_db", index_name=("WW2"))

In [49]:
from langchain import HuggingFaceHub

llm=HuggingFaceHub(huggingfacehub_api_token="",
                   repo_id="tiiuae/falcon-7b", #
                    model_kwargs={"temperature":0.5 ,
                                 "max_length":512,
                                 "max_new_tokens":200
                                 })

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [68]:
from langchain.chains import RetrievalQA
from langchain.schema import retriever
from langchain.memory import ConversationBufferMemory
from langchain.chains.question_answering import load_qa_chain


vector_db=FAISS.load_local(folder_path="./FAISS_db",
                           index_name='WW2',
                           embeddings=embeddings)

memory = ConversationBufferMemory(memory_key="chat_history")
qa_chain=load_qa_chain(llm=llm, chain_type="stuff")
qa = RetrievalQA(combine_documents_chain=qa_chain,
                                 retriever=vector_db.as_retriever(),
                                 memory=memory,
                                )

In [75]:
#Ask a question?
queary='Wha were the reasons of WWII?'
results=qa({"query": queary})

In [76]:
results['chat_history']

'Human: When Germany invaded in USSR?\nAI: \n\nThe Germans invaded the USSR on June 22, 1941, and the Soviets declared war on them on\nSeptember 1, 1941.\n\nThe Battle of Stalingrad\n\nThe Battle of Stalingrad was a turning point in the war.\nThe Germans were defeated and forced to retreat.\nThe battle was fought between September 1942 and February 1943.\nThe Axis Powers lost 1 million men and the Soviets lost 900,000 men.\nThe Battle of Stalingrad was a turning point in the war.\nThe Battle of Stalingrad was a turning point in the war.\nThe Battle of Stalingrad was a turning point in the war.\nThe Battle of Stalingrad was a turning point in the war.\nThe Battle of Stalingrad was a turning point in the war.\n\nFurther, the Soviets lost 900,000 men and the Axis Powers lost 1 million\nHuman: When USA did take a part in WWII?\nAI: \n\nThe United States entered World War II after the Japanese attack on Pearl Harbor on 7 December 1941.\nThe United States had been neutral until then.\nThe US

In [77]:
results['result']

'\nThe major causes of World War II were numerous. They include the impact of the Treaty of\nVersailles following WWI, the worldwide economic depression, failure of appeasement, the\nrise of militarism in Germany and Japan, and the failure of the League of Nations.\n\nThe Treaty of Versailles\nFollowing World War I, the victorious Allied Powers met to decide Germany’s future. Germany\nwas forced to sign the Treaty of Versailles.\nUnder this treaty, Germany had to accept guilt for the war and to pay reparations. Germany lost\nterritory and was prohibited from having a large military.\nThe humiliation faced by Germany under this treaty paved the way for the spread of Ultra-\nNationalism in Germany.\n\nFailure of the League of Nations\nThe League of Nations was an international organization set up in 1919 to keep world peace.\nIt was intended that all countries would be members and that if there were disputes between\ncount'

In [38]:
results.keys()

dict_keys(['query', 'chat_history', 'result'])